## 라이브러리

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [14]:
import os
path = '../script'

# 다큐멘터리(0), 영화(1) 선택
category = 2
if category == 0:
    path = path + '/Documentary'
elif category == 1:
    path = path + '/Movie'

_list = os.listdir(path)
file_list = []
for file in _list:
    if file.startswith('.'): continue
    file_list.append(file)

file_list = sorted(file_list)
print(f'작품개수: {len(file_list)}')
# print([{i:_list[i]} for i in range(len(file_list))])
for index, title in enumerate(file_list):
    print(f'{index}: {title}')

작품개수: 46
0: About.Time
1: Arthur.Christmas
2: Baby.Driver
3: Baby.DriverBaby.Driver
4: Bad.Boys
5: Bee.Movie
6: Ben.Platt.Live.from.Radio.City.Music.Hall
7: Ben.Platt.Live.from.Radio.City.Music.HallBen.Platt.Live.from.Radio.City.Music.Hall
8: Constantine
9: Dolittle
10: Dracula.Untold
11: Enola.Holmes
12: How.to.Be.Single
13: In.Time
14: Inception
15: Interstellar
16: It's.Complicated
17: Jumanji_.Welcome.to.the.Jungle
18: Kill.Bill_.Vol..1
19: Klaus
20: Kung.Fu.Panda
21: Kung.Fu.Panda.2
22: Kung.Fu.Panda.3
23: Madagascar_.Escape.2.Africa
24: Monty.Python.and.the.Holy.Grail
25: Morning.Glory
26: Notting.Hill
27: Passengers
28: Penguins.of.Madagascar_.The.Movie
29: Prisoners
30: PrisonersPrisoners
31: Second.Act
32: The.Accountant
33: The.Boy.in.the.Striped.Pajamas
34: The.Dark.Knight
35: The.Dark.KnightThe.Dark.Knight
36: The.Departed
37: The.Family.Man
38: The.Family.ManThe.Family.Man
39: The.Intern
40: The.Judge
41: The.Revenant
42: The.Truman.Show
43: There.Will.Be.Blood
44: There.W

## 여기서 작품 선택합니다 꼭!!

In [18]:
# 바로 위 코드에서 file_list를 참고하여 작품 선택
title = file_list[0]

file_name = title + '.WEBRip.Netflix.en[cc]'
print(f'{path}/{title}/{file_name}')

../script/Movie/About.Time/About.Time.WEBRip.Netflix.en[cc]


## VTT 파일 읽기

In [19]:
opened_file = open(f'{path}/{title}/{file_name}.vtt', encoding='utf8')
content = opened_file.read()
segments = content.split('\n\n')    # 한줄씩 정리

In [20]:
segments[:5]

['WEBVTT',
 'NOTE Netflix\nNOTE Profile: webvtt-lssdh-ios8\nNOTE Date: 2015/07/27 20:25:13',
 'NOTE SegmentIndex\nNOTE Segment=596.458 55824@734 339\nNOTE Segment=595.334 52258@56558 323\nNOTE Segment=570.000 57352@108816 355\nNOTE Segment=588.000 53460@166168 332\nNOTE Segment=596.042 50974@219628 314\nNOTE Segment=582.083 66848@270602 411\nNOTE Segment=595.917 53618@337450 332\nNOTE Segment=597.041 55656@391068 339\nNOTE Segment=586.917 52088@446724 319\nNOTE Segment=596.875 55484@498812 343\nNOTE Segment=593.416 45924@554296 278\nNOTE Segment=596.792 38542@600220 233\nNOTE Segment=145.667 5588@638762 32\nNOTE /SegmentIndex',
 '                                                                                                 ',
 '                    \n1\n00:00:29.834 --> 00:00:32.917  position:19.52%,start  align:start size:44.29%  line:84.62% \n<c.white><c.mono_sans>[ Waves Crashing ]</c.mono_sans></c.white>']

## 불필요한 부분 제거

In [21]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [22]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:5]

[None,
 None,
 None,
 '4\n00:00:54.291  00:00:57.208  I always knew',
 '5\n00:00:54.291  00:00:57.208  we were a fairly odd family.']

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [23]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment.strip() != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = ' '.join(split_segment[3:])
      times.append(script)
      trimmed_segments.append(times)

In [24]:
trimmed_segments[:5]

[['4', '00:00:54.291', '00:00:57.208', 'I always knew'],
 ['5', '00:00:54.291', '00:00:57.208', 'we were a fairly odd family.'],
 ['6', '00:00:57.250', '00:01:02.125', 'First there was me.'],
 ['7', '00:00:57.250', '00:01:02.125', 'Too tall, too skinny,'],
 ['8', '00:00:57.250', '00:01:02.125', 'too orange.']]

## 데이터 저장

In [25]:
with open(f'{path}/{title}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [26]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

data.head()

,num,start,end,script
0,4,00:00:54.291,00:00:57.208,I always knew
1,5,00:00:54.291,00:00:57.208,we were a fairly odd family.
2,6,00:00:57.250,00:01:02.125,First there was me.
3,7,00:00:57.250,00:01:02.125,"Too tall, too skinny,"
4,8,00:00:57.250,00:01:02.125,too orange.


In [27]:
data['script'] = data['script'].str.replace('\"', '')

# 쪼개진 문장 합치기
for index, current_row in data.iterrows():
  if current_row['script'][-1] in ['.', '!', '?']:
    continue
  else:
    row_index = index + 1
    while True:
      next_row = data.loc[row_index]
      new_script = data.loc[index, 'script'] + ' ' + next_row['script']
      data.loc[index, 'script'] = new_script
      if next_row['script'][-1] in ['.', '!', '?']:
        data.loc[index, 'end'] = next_row['end']
        break
      else:
        row_index += 1

# 필요없는 토막문장들 제거
data = data.drop_duplicates(subset='end')
data.head(30)

,num,start,end,script
0,4,00:00:54.291,00:00:57.208,I always knew we were a fairly odd family.
2,6,00:00:57.250,00:01:02.125,First there was me.
5,9,00:01:02.166,00:01:05.083,"My mum was lovely, but not like other mums."
7,11,00:01:05.125,00:01:09.208,"There was something solid about her-- something rectangular, busy and unsentimental."
10,14,00:01:09.250,00:01:11.000,Her fashion icon was the queen.
12,16,00:01:12.083,00:01:14.667,"Dad-- Well, he was more normal."
13,17,00:01:14.709,00:01:16.834,He always seemed to have time on his hands.
15,19,00:01:16.875,00:01:22.417,"After giving up teaching university students on his 50th birthday, he was eternally available for a leisurely chat..."
20,24,00:01:22.458,00:01:24.709,or to let me win at table tennis.
22,26,00:01:24.750,00:01:26.667,- Ah!


In [56]:
data.to_csv(f'{path}/{title}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv')

data.head()

,start,end,script
0,00:01:09.569,00:01:12.447,"BEN: Freud said, Love and work. Work and love."
1,00:01:12.614,00:01:14.324,That's all there is.
2,00:01:14.491,00:01:17.327,"Well, I'm retired, and my wife is dead."
3,00:01:17.494,00:01:20.663,"As you can imagine, that's given me some time on my hands."
4,00:01:21.164,00:01:23.416,My wife's been gone for three and a half years.
